In [66]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import librosa
import soundfile as sf

In [67]:

queen_training_file = "QueenBee_Training_45mins.wav"
queen_training_data, queen_sampling_rate = librosa.load(queen_training_file, sr=None)  # sr=None to get the original sampling rate

# resample for no queen part
no_queen_training_file = "No_QueenBee_Training_45mins.wav"
no_queen_training_data, no_queen_sampling_rate = librosa.load(no_queen_training_file, sr=None)  # sr=None to get the original sampling rate

#sample_rate, audio = librosa.load("beeAudioFile.mp3")
print("queen_sampling_rate: {0}Hz".format(queen_sampling_rate))
print("no_queen_sampling_rate: {0}Hz".format(no_queen_sampling_rate))
segment_length = 3  # seconds

segment_length_frames = int(segment_length * queen_sampling_rate)

# Create segments for queen and no queen training data
segment_length = 3  # seconds
queen_training_data_segments = [
                                queen_training_data[i:i + segment_length_frames] 
                                    for i in range(0, len(queen_training_data), segment_length_frames)
                                ]

no_queen_training_data_segments =  [
                                no_queen_training_data[i:i + segment_length_frames] 
                                    for i in range(0, len(no_queen_training_data), segment_length_frames)
                                    ]

# Ensure you have 900 segments for each class
print("Number of Queen Segments:", len(queen_training_data_segments))
print("Number of No Queen Segments:", len(no_queen_training_data_segments))

# Assuming queen_training_data_segments and no_queen_training_data_segments are your original audio segments
# Extract MFCC features

queen_mfccs = []  
queen_labels = []
no_queen_mfccs = [] 
no_queen_labels = []

# Assign labels for Queen class
for segment in queen_training_data_segments:
    queen_mfcc = librosa.feature.mfcc(y=segment, sr=queen_sampling_rate, n_mfcc=13)
    queen_mfccs.append(queen_mfcc)
    queen_labels.append(1)  # Assign label 1 for Queen class

# Assign labels for No Queen class
for segment in no_queen_training_data_segments:
    no_queen_mfcc = librosa.feature.mfcc(y=segment, sr=queen_sampling_rate, n_mfcc=13)
    no_queen_mfccs.append(no_queen_mfcc)
    no_queen_labels.append(0)  # Assign label 0 for No Queen class

# Calculate the minimum length among all segments
min_length = min(mfcc.shape[1] for mfcc in queen_mfccs + no_queen_mfccs)

# Trim MFCC matrices to the minimum length
queen_mfccs = [mfcc[:, :min_length] for mfcc in queen_mfccs]
no_queen_mfccs = [mfcc[:, :min_length] for mfcc in no_queen_mfccs]

# Check the number of columns in each array
queen_numberOfColumn = set(mfcc.shape[1] for mfcc in queen_mfccs)
no_queen_numberOfColumn = set(mfcc.shape[1] for mfcc in no_queen_mfccs)

print("Queen MFCC Number Of Column:", queen_numberOfColumn)
print("No Queen MFCC Number Of Column:", no_queen_numberOfColumn)
print("MFCC Matrix Shape:", queen_mfccs[0].shape)

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Combine the data from "queen" and "no queen" segments and labels:
X = np.vstack((queen_mfccs, no_queen_mfccs))
y = np.hstack((queen_labels, no_queen_labels))

# Reshape X to have two dimensions
X = X.reshape(X.shape[0], -1)  # Flatten the last two dimensions

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier (you can choose different kernels and parameters)
clf = svm.SVC(kernel='linear')

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy and print the classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of MFCC Features: {accuracy * 100:.2f}%")

report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)



queen_sampling_rate: 44100Hz
no_queen_sampling_rate: 44100Hz
Number of Queen Segments: 929
Number of No Queen Segments: 924
Queen MFCC Number Of Column: {152}
No Queen MFCC Number Of Column: {152}
MFCC Matrix Shape: (13, 152)
Accuracy of MFCC Features: 99.19%

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       185
           1       0.99      0.99      0.99       186

    accuracy                           0.99       371
   macro avg       0.99      0.99      0.99       371
weighted avg       0.99      0.99      0.99       371



extract MFCC features and label

Queen MFCC Lengths: {152, 259}
No Queen MFCC Lengths: {259, 159}


variations in the number of frames (MFCC lengths) across different segments within the same class can be considered a normal phenomenon, especially when dealing with audio data. 

In [77]:
queen_validation_file = "QueenBee_Testing_15mins.wav"
no_queen_validation_file = "No_QueenBee_Testing_15mins.wav"

queen_validation_data, queen_validation_sampling_rate = librosa.load(queen_validation_file, sr=None)
no_queen_validation_data, no_queen_validation_sampling_rate = librosa.load(no_queen_validation_file, sr=None)



segment_length = 3  # seconds

segment_length_frames = int(segment_length * queen_validation_sampling_rate)

# Create segments for queen and no queen training data
segment_length = 3  # seconds
queen_validation_data_segments = [
                                queen_validation_data[i:i + segment_length_frames] 
                                    for i in range(0, len(queen_validation_data), segment_length_frames)
                                ]

no_queen_validation_data_segments =  [
                                no_queen_validation_data[i:i + segment_length_frames] 
                                    for i in range(0, len(no_queen_validation_data), segment_length_frames)
                                    ]

# Ensure you have 900 segments for each class
print("Number of Queen Segments:", len(queen_validation_data_segments))
print("Number of No Queen Segments:", len(no_queen_validation_data_segments))


queen_validation_mfccs = []  
queen_validation_labels = []
no_queen_validation_mfccs = [] 
no_queen_validation_labels = []
target_number_of_columns = 152

# Assign labels for Queen class
for segment in queen_validation_data_segments:
    queen_mfcc = librosa.feature.mfcc(y=segment, sr=queen_validation_sampling_rate, n_mfcc=13)
    
    queen_validation_mfcc_resampled = queen_mfcc[:, :target_number_of_columns]
    if queen_validation_mfcc_resampled.shape[1] == target_number_of_columns:
        queen_validation_mfccs.append(queen_validation_mfcc_resampled)
        queen_validation_labels.append(1)  # Assign label 1 for Queen class
    
# Assign labels for No Queen class
for segment in no_queen_validation_data_segments:
    no_queen_mfcc = librosa.feature.mfcc(y=segment, sr=queen_validation_sampling_rate, n_mfcc=13)
    no_queen_validation_mfcc_resampled = no_queen_mfcc[:, :target_number_of_columns]
    no_queen_validation_mfccs.append(no_queen_validation_mfcc_resampled)
    no_queen_validation_labels.append(0)  # Assign label 0 for No Queen class

# Check the number of columns in each array
queen_mfcc_numberOfColumn = set(mfcc.shape[1] for mfcc in queen_validation_mfccs)
no_queen_mfcc_numberOfColumn = set(mfcc.shape[1] for mfcc in no_queen_validation_mfccs)

print("Queen MFCC Number Of Column:", queen_mfcc_numberOfColumn)
print("No Queen MFCC Number Of Column:", no_queen_mfcc_numberOfColumn)
print("MFCC Matrix Shape:", queen_validation_mfccs[0].shape)



# Trim the MFCC matrices to the target number of columns

no_queen_validation_mfcc_resampled = no_queen_validation_mfcc_resampled[:, :target_number_of_columns]



Number of Queen Segments: 314
Number of No Queen Segments: 324
Queen MFCC Number Of Column: {152}
No Queen MFCC Number Of Column: {152}
MFCC Matrix Shape: (13, 152)


In [80]:
# Combine the data from "queen" and "no queen" segments and labels:
X_validation = np.vstack((queen_validation_mfccs, no_queen_validation_mfccs))
y_validation = np.hstack((queen_validation_labels, no_queen_validation_labels))

# Assuming X_train is your training data
print("Number of features in training data:", X_train.shape[1])

# Assuming X_test is your validation data
print("Number of features in validation data:", X_validation.shape[1])


# Reshape X to have two dimensions
X_validation = X_validation.reshape(X_validation.shape[0], -1)  # Flatten the last two dimensions

# Make predictions on the test data
y_pred = clf.predict(X_validation)


# Calculate accuracy and print the classification report
accuracy = accuracy_score(y_validation, y_pred)
print(f"Accuracy of MFCC Features: {accuracy * 100:.2f}%")

report = classification_report(y_validation, y_pred)
print("\nClassification Report:\n", report)

Number of features in training data: 1976
Number of features in validation data: 13
Accuracy of MFCC Features: 49.14%

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       324
           1       0.49      1.00      0.66       313

    accuracy                           0.49       637
   macro avg       0.25      0.50      0.33       637
weighted avg       0.24      0.49      0.32       637



c:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
